In [12]:
'''imports'''
'''This file is used for the prediction of stock prices based upon the dataset located here:
    https://www.kaggle.com/datasets/borismarjanovic/price-volume-data-for-all-us-stocks-etfs 
    data import, model training, and model testing will all be included in this file
 '''
#main imports 
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [13]:
'''importing the data'''
df = pd.read_csv('../Datasets/StockPricesData/Stocks/a.us.txt')

df = df.drop(['Date','OpenInt'], axis=1)

df.head()
print(df.shape)
# #train test split
size = df.shape[0]
size80 = int(size*0.8)

dfTrain = df.iloc[:size80] #grabs the train section of the data
dfTest = df.iloc[size80:] #grabs the test data



(4521, 5)


In [14]:
#normalizing the data
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
dfTrain = sc.fit_transform(dfTrain)

In [22]:
#creating batches of time and the labels for each batch
xTrain = []
yTrain = []
for i in range(60, len(dfTrain)-1):
    #,0 changes the dimension of the array to 1d
    xTrain.append(dfTrain[i-60:i, 0])
    yTrain.append(dfTrain[i, 0])
    
xTrain, yTrain = np.array(xTrain), np.array(yTrain)
xTrain = np.reshape(xTrain, (xTrain.shape[0], xTrain.shape[1], 1))
xTrain.shape


(3555, 60, 1)

In [16]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense

model = Sequential()
model.add(LSTM(units=50,return_sequences=True,input_shape=(xTrain.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer='adam',loss='mean_squared_error')
model.fit(xTrain,yTrain,epochs=100,batch_size=32)





Epoch 1/100

112/112 [==============================] - 9s 41ms/step - loss: 0.0030
Epoch 2/100
112/112 [==============================] - 4s 39ms/step - loss: 0.0013
Epoch 3/100
112/112 [==============================] - 4s 39ms/step - loss: 0.0012
Epoch 4/100
112/112 [==============================] - 5s 43ms/step - loss: 0.0010
Epoch 5/100
112/112 [==============================] - 5s 42ms/step - loss: 8.5096e-04
Epoch 6/100
112/112 [==============================] - 6s 51ms/step - loss: 8.8017e-04
Epoch 7/100
112/112 [==============================] - 5s 48ms/step - loss: 8.9711e-04
Epoch 8/100
112/112 [==============================] - 5s 48ms/step - loss: 8.4057e-04
Epoch 9/100
112/112 [==============================] - 6s 53ms/step - loss: 6.7313e-04
Epoch 10/100
112/112 [==============================] - 5s 47ms/step - loss: 7.9532e-04
Epoch 11/100
112/112 [==============================] - 6s 52ms/step - loss: 5.9854e-04
Epoch 12/100
112/112 [==============================] 